In [1]:
import open3d as o3d
import numpy as np
import json

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
with open("/workspace/chat-with-nerf-eval/data/scannet/scans/scene0025_00/outputs/exported/lerf/2023-09-04_065104/dataparser_transforms.json", "r") as file:
    data = json.load(file)

In [3]:
transform = np.array(data["transform"])
scale = data["scale"]

In [4]:
rotation = transform[:, :3]
translation = transform[:, 3]

rotation_inv = rotation.T
translation_inv = -np.dot(rotation_inv, translation)

In [5]:
rotation_inv

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [6]:
translation_inv

array([2.63335919, 3.252738  , 1.54422593])

In [7]:
pcd = o3d.io.read_point_cloud("/workspace/chat-with-nerf-eval/data/scannet/scans/scene0025_00/pcd_results/point_cloud.ply")

# 1. Apply the inverse scale
pcd.points = o3d.utility.Vector3dVector(np.asarray(pcd.points) / scale)

# 2. Apply the inverse transformation
points_transformed = np.dot(np.asarray(pcd.points), rotation_inv) + translation_inv
pcd.points = o3d.utility.Vector3dVector(points_transformed)

# Save the transformed point cloud if needed
o3d.io.write_point_cloud("/workspace/chat-with-nerf-eval/data/scannet/scans/scene0025_00/pcd_results/transformed_point_cloud.ply", pcd)

True

## cutoff points

In [8]:
import open3d as o3d

In [9]:
# Load your point cloud
pcd = o3d.io.read_point_cloud("/workspace/chat-with-nerf-eval/data/scannet/scans/scene0025_00/pcd_results/transformed_point_cloud.ply")

# Load your mesh
mesh = o3d.io.read_triangle_mesh("/workspace/chat-with-nerf-eval/data/scannet/scans/scene0025_00/scene0025_00_vh_clean_2.ply")

In [14]:
obb = mesh.get_oriented_bounding_box()

In [21]:
R = obb.R
t = obb.center
T = np.eye(4)
T[:3, :3] = R
T[:3, 3] = t

In [16]:
points_homogeneous = np.hstack((np.asarray(pcd.points), np.ones((len(pcd.points), 1))))

In [23]:
inv_transform = np.linalg.inv(T)

In [25]:
T

array([[ 0.64491218, -0.76388451, -0.02384796,  3.02414145],
       [-0.76224071, -0.64516277,  0.05247944,  2.85866997],
       [-0.05547405, -0.01566674, -0.99833721,  1.25797859],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [26]:
transformed_points = (inv_transform @ points_homogeneous.T).T

In [27]:
obb_half_extent = obb.extent / 2

In [28]:
inside_mask = np.all(np.abs(transformed_points[:, :3]) <= obb_half_extent, axis=1)


In [32]:
inside_points = o3d.utility.Vector3dVector(np.asarray(pcd.points)[inside_mask])
inside_colors = o3d.utility.Vector3dVector(np.asarray(pcd.colors)[inside_mask]) # Added this line


In [33]:
inside_pcd = o3d.geometry.PointCloud(points=inside_points)
inside_pcd.colors = inside_colors # Added this line


In [34]:
o3d.io.write_point_cloud("/workspace/chat-with-nerf-eval/data/scannet/scans/scene0025_00/pcd_results/inside_pcd.ply", inside_pcd)

True